In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats
import re
import graphlab as gl
from graphlab.toolkits.feature_engineering import TFIDF, FeatureHasher, QuadraticFeatures

gl.canvas.set_target('ipynb')

PATH_TO_JSON2 = "/mnt/sframe/docs_prod_02/"
PATH_TO_JSON = "/mnt/sframe/docs_prod_05/"
PATH_TO_JSON6 = "/mnt/sframe/docs_prod_06/"
PATH_TO_JSON7 = "/mnt/sframe/docs_prod_07/"
PATH_TO_JSON8 = "/mnt/sframe/docs_prod_08/"
PATH_TO_TRAIN_LABELS = "input/train.csv"
PATH_TO_TEST_LABELS = "input/sampleSubmission.csv"

### Read processed documents

In [2]:
gl.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 128)
gl.set_runtime_config('GRAPHLAB_FILEIO_MAXIMUM_CACHE_CAPACITY', 100*1024*1024*1024) # 100GB
gl.set_runtime_config('GRAPHLAB_FILEIO_MAXIMUM_CACHE_CAPACITY_PER_FILE', 100*1024*1024*1024) # 100GB

[INFO] This non-commercial license of GraphLab Create is assigned to renatbek@gmail.comand will expire on October 09, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-14059 - Server binary: /usr/local/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1444374132.log
[INFO] GraphLab Server Version: 1.6.1


In [3]:
def transf(x):
    return x
    #return 50.0 * np.log1p(np.log1p(x))

In [4]:
# documents
sf = gl.SFrame.read_csv(PATH_TO_JSON, header=False, verbose=False)
sf = sf.unpack('X1',column_name_prefix='')
sf['id'] = sf['id'].apply(lambda x: str(x.split('_')[0] ))
sf['num_words'] = sf['text'].apply(lambda xs: transf(len(xs)))

In [5]:
sf2 = gl.SFrame.read_csv(PATH_TO_JSON2, header=False, verbose=False)
sf2 = sf2.unpack('X1',column_name_prefix='')
sf2['id'] = sf2['id'].apply(lambda x: str(x.split('_')[0] ))

RuntimeError: Runtime Exception. CSV parsing cancelled

In [ ]:
sf6 = gl.SFrame.read_csv(PATH_TO_JSON6, header=False, verbose=False)
sf6 = sf6.unpack('X1',column_name_prefix='')
sf6['id'] = sf6['id'].apply(lambda x: str(x.split('_')[0] ))
sf6['word2vec'] = sf6['word2vec'].apply(lambda xs: np.array(xs ,dtype='float32').tolist())

In [ ]:
sf7 = gl.SFrame.read_csv(PATH_TO_JSON7, header=False, verbose=False)
sf7 = sf7.unpack('X1',column_name_prefix='')
sf7['id'] = sf7['id'].apply(lambda x: str(x.split('_')[0] ))

In [6]:
sf8 = gl.SFrame.read_csv(PATH_TO_JSON8, header=False, verbose=False)
sf8 = sf8.unpack('X1',column_name_prefix='')
sf8['id'] = sf8['id'].apply(lambda x: str(x.split('_')[0] ))
sf8_orig = sf8

In [57]:
sf8 = sf8_orig.unpack('viewport')

sf8 = sf8.unpack('links_profile')
sf8 = sf8.fillna('links_profile.candidate_site', '')
sf8['links_profile.candidate_site'] = sf8['links_profile.candidate_site'].apply(lambda s: len(s) if s and s != 'null' else 0)

sf8['meta_names'] = sf8['meta_names'].apply(lambda s: ','.join(s))

sf8 = sf8.remove_column('links_profile.domains')

In [58]:
sf8.head()

id,meta_names,viewport.0,viewport.1,viewport.2,links_profile.bad_domain,links_profile.candidate_rank ...
3450965,,0,0,0,0,0
3450935,generator,0,0,0,1,409377
345089,"description,referrer,twitter:dnt,fb:app_id,og: ...",0,1,0,2,0
3450887,"apple-itunes-app,viewport,description,keywords ...",1,1,1,11,12393
3450851,"twitter:site,twitter:site:id,twitter:creator,t ...",1,1,0,36,0
3450803,"description,og:locale,og:type,og:title,og:desc ...",1,1,1,19,152465
3450743,"description,keywords",0,0,0,0,0
3450695,"charset,og:title,twitter:title,msapplication-tap- ...",1,1,0,0,0
3450671,"description,og:description,og:image,og:title,o ...",0,0,0,0,0
3450623,"viewport,description,viewport ...",1,1,0,10,0


In [46]:
sf8['links_profile.candidate_sit'].show()

In [37]:
sf8.show()

In [ ]:
sf_cnt = gl.SFrame()
sf_cnt['id'] = sf2['id']

sf_cnt['a_href'] = sf2['ahref'].apply(lambda x: transf(len(x)))
sf_cnt['par'] = sf2['par'].apply(lambda x: transf(len(x)))
sf_cnt['title'] = sf2['title'].apply(lambda x: transf(len(x)))

sf_cnt['img'] = sf2['img_cnt'].apply(transf)
sf_cnt['btn'] = sf2['misc_button'].apply(transf)
sf_cnt['input'] = sf2['misc_input'].apply(transf)
sf_cnt['li'] = sf2['misc_li'].apply(transf)
sf_cnt['link'] = sf2['misc_link'].apply(transf)
sf_cnt['meta'] = sf2['misc_meta'].apply(transf)

sf_cnt['script_avg'] = sf2['script_avg'].apply(transf)
sf_cnt['script_b_avg'] = sf2['script_b_avg'].apply(transf)
sf_cnt['script_cnt'] = sf2['script_cnt'].apply(transf)
sf_cnt['script_b_cnt'] = sf2['script_b_cnt'].apply(transf)

sf_cnt['style_avg'] = sf2['style_avg'].apply(transf)
sf_cnt['style_cnt'] = sf2['style_cnt'].apply(transf)

### Read train/test labels and merge into documents

In [ ]:
# train/test labels
train_labels = gl.SFrame.read_csv(PATH_TO_TRAIN_LABELS, verbose=False)
test_labels = gl.SFrame.read_csv(PATH_TO_TEST_LABELS, verbose=False)
train_labels['id'] = train_labels['file'].apply(lambda x: str(x.split('_')[0] ))
train_labels = train_labels.remove_column('file')
test_labels['id'] = test_labels['file'].apply(lambda x: str(x.split('_')[0] ))
test_labels = test_labels.remove_column('file')

In [ ]:
# join
train = train_labels.join(sf, on='id', how='left')
test = test_labels.join(sf, on='id', how='left')

In [ ]:
train = train.join(sf_cnt, on='id', how='left')
test = test.join(sf_cnt, on='id', how='left')

In [ ]:
train = train.join(sf6, on='id', how='left')
test = test.join(sf6, on='id', how='left')

In [ ]:
train = train.join(sf7, on='id', how='left')
test = test.join(sf7, on='id', how='left')

In [60]:
train = train.join(sf8, on='id', how='left')
test = test.join(sf8, on='id', how='left')

In [61]:
train.shape, test.shape, sf8.shape

((337024, 65), (66772, 65), (403618, 13))

In [65]:
train.print_rows(num_columns=65)

+-----------+--------+-------------------------------+-----------+--------+-----+
| sponsored |   id   |              text             | num_words | a_href | par |
+-----------+--------+-------------------------------+-----------+--------+-----+
|     0     | 920982 | bill gate world richest mo... |    2670   |   56   |  9  |
|     0     | 920988 | credit netflix fandom leah... |    6671   |  183   |  20 |
|     0     | 920992 |  bimota microsoft frontpag... |     52    |   0    |  0  |
|     0     | 921036 | make your reflect your poi... |    6962   |  227   |  1  |
|     0     | 921040 | play music suppos rememb s... |    5177   |  110   |  22 |
|     0     | 921050 | ecogrid ecorast ecogrid ec... |    2012   |   74   |  16 |
|     0     | 921108 | topic sunday polic charg y... |    7318   |   70   |  38 |
|     1     | 921123 | celebr your passion hobbi ... |    3566   |   19   |  21 |
|     1     | 921127 |  local queri search langua... |    260    |   6    |  0  |
|     0     | 92

In [ ]:
f = 'ga_len'
train = train.fillna(f, 1000)     
test = test.fillna(f, 1000)
f = 'ga_line'
train = train.fillna(f, 5000)     
test = test.fillna(f, 5000)
f = 'basic'
train = train.fillna(f, [0,0,0,0,0,0])     
test = test.fillna(f, [0,0,0,0,0,0])

In [ ]:
features = [
            'a_href',
            'par',
            'title',
            'img',
            'btn',
            'input',
            'li',
            'link',
            'meta',
            'script_avg',
            'script_b_avg',
            'script_cnt',
            'script_b_cnt',
            'style_avg',
            'style_cnt',
            'num_words',
            'fb_id', # new features GA FB WP
            'ga_id',
            'ga_subid',
            'ga_cnt',
            'ga_len',
            'ga_line',
           ]

In [ ]:
for f in features:
    train = train.fillna(f, 0.0)     
    test = test.fillna(f, 0.0)

In [ ]:
train['ga_id'] = train['ga_id'].apply(float)
test['ga_id'] = test['ga_id'].apply(float)

train['ga_subid'] = train['ga_subid'].apply(float)
test['ga_subid'] = test['ga_subid'].apply(float)

train['fb_id'] = train['fb_id'].apply(float)
test['fb_id'] = test['fb_id'].apply(float)

In [ ]:
features = features + ['basic']

In [ ]:
train = train.fillna('shinn', {})     
test = test.fillna('shinn', {})

train['shinn'] = train['shinn'].apply(lambda ws: ws if ws else {})
test['shinn'] = test['shinn'].apply(lambda ws: ws if ws else {})

features = features + ['shinn']

In [ ]:
train = train.fillna('word2vec', np.zeros(300))     
test = test.fillna('word2vec', np.zeros(300))

train['word2vec'] = train['word2vec'].apply(lambda ws: ws if ws else np.zeros(300))
test['word2vec'] = test['word2vec'].apply(lambda ws: ws if ws else np.zeros(300))

In [ ]:
train = train.fillna('words', [])     
test = test.fillna('words', [])   

train['words'] = train['words'].apply(lambda ws: ws if ws else [])
test['words'] = test['words'].apply(lambda ws: ws if ws else [])

In [ ]:
train['word_set_size'] = train['words'].apply(lambda ws: len(set(ws)))
test['word_set_size'] = test['words'].apply(lambda ws: len(set(ws)))

train['word_set_size_ratio'] = train.apply(lambda r: r['word_set_size'] * 1.0 / len(r['words']) if len(r['words']) > 0 else 0.0)
test['word_set_size_ratio'] = test.apply(lambda r: r['word_set_size'] * 1.0 / len(r['words']) if len(r['words']) > 0 else 0.0)

In [ ]:
train['text_words'] = train['words'].apply(lambda ws: ' '.join(ws))
test['text_words'] = test['words'].apply(lambda ws: ' '.join(ws))

In [ ]:
features = features + ['word_set_size', 'word_set_size_ratio']

In [ ]:
def text2wordsetsize(text):
    if not text:
        return 0.0
    else:
        return len(set(text.split()))
    
def safe_divide(a, b):
    if b > 0:
        return a * 1.0 / b
    else:
        return 0.0

train['word_set_size2'] = train['text'].apply(text2wordsetsize)
test['word_set_size2'] = test['text'].apply(text2wordsetsize)

train['word_set_size2_ratio'] = train.apply(lambda r: safe_divide(r['word_set_size2'], r['word_set_size']))
test['word_set_size2_ratio'] = test.apply(lambda r: safe_divide(r['word_set_size2'], r['word_set_size']))

features = features + ['word_set_size2', 'word_set_size_ratio2']

In [ ]:
train.head()

In [ ]:
#train.save('/mnt/sframe/sframe_train')
#test.save('/mnt/sframe/sframe_test')

In [ ]:
train.remove_column('text_words')
test.remove_column('text_words')
train.remove_column('words')
test.remove_column('words')
train.remove_column('word2vec')
test.remove_column('word2vec')

In [ ]:
train = train.unpack('shinn')
test = test.unpack('shinn')

train = train.unpack('basic')
test = test.unpack('basic')

In [67]:
df_train = train.to_dataframe()
df_test = test.to_dataframe()

df_train.to_pickle('/mnt/sframe/df_train')
df_test.to_pickle('/mnt/sframe/df_test')

In [66]:
df_train.shape

(337024, 53)

In [ ]:
train = train.pack_columns(column_prefix='shinn', dtype=dict, new_column_name='shinn')
test = test.pack_columns(column_prefix='shinn', dtype=dict, new_column_name='shinn')

train = train.pack_columns(column_prefix='basic', dtype=dict, new_column_name='basic')
test = test.pack_columns(column_prefix='basic', dtype=dict, new_column_name='basic')

In [ ]:
# Retrieve train/test
df_train = pd.read_pickle('/mnt/sframe/df_train_noviewport')
df_test = pd.read_pickle('/mnt/sframe/df_test_noviewport')

In [55]:
train = gl.SFrame(df_train)
test = gl.SFrame(df_test)

In [56]:
train.head()

sponsored,id,text,num_words,a_href,par,title,img,btn,input,li,link
0,920982,bill gate world richestmost time beaten norweg ...,2670,56,9,44,4,0,4,30,13
0,920988,credit netflix fandomleah tedesco follow ...,6671,183,20,31,13,0,12,150,10
0,920992,bimota microsoftfrontpag frontpag editor ...,52,0,0,12,0,0,0,0,0
0,921036,make your reflect yourpoint across with thi ...,6962,227,1,73,23,13,15,81,30
0,921040,play music suppos remembsometim electron ...,5177,110,22,67,44,0,3,68,5
0,921050,ecogrid ecorast ecogridecorast ecogrid ecorast ...,2012,74,16,75,12,0,0,15,8
0,921108,topic sunday polic chargyear jeffrey william ...,7318,70,38,68,13,4,13,24,11
1,921123,celebr your passion hobbiwith great video more ...,3566,19,21,60,5,0,0,26,23
1,921127,local queri searchlanguag countri welcom ...,260,6,0,44,0,0,2,0,4
0,921130,they necess mother inventwhat they neglect men ...,12831,210,26,65,37,1,10,158,19


### Generate BOW

In [ ]:
bow_trn = gl.text_analytics.count_words(train['text'])
bow_trn = bow_trn.dict_trim_by_keys(gl.text_analytics.stopwords())

bow_tst = gl.text_analytics.count_words(test['text'])
bow_tst = bow_tst.dict_trim_by_keys(gl.text_analytics.stopwords())

train['bow'] = bow_trn
test['bow'] = bow_tst

### Generate TF-IDF

In [ ]:
encoder = gl.feature_engineering.create(train, TFIDF('bow', output_column_name='tfidf', min_document_frequency=5e-5))
train = encoder.transform(train)
test = encoder.transform(test)

In [ ]:
train['tfidf'] = train['tfidf'].fillna({})
test['tfidf'] = test['tfidf'].fillna({})

train['tfidf'] = train['tfidf'].apply(lambda x: x if x else {})
test['tfidf'] = test['tfidf'].apply(lambda x: x if x else {})

## Features

In [ ]:
def feature_col(col):
    return col not in set(['sponsored', 'id', 'text', 'text_words', 'word2vec', 'words', 'bow'])

features = [f for f in train.column_names() if feature_col(f)]

### Split training set for cross-validation

In [ ]:
train_train, train_cv = train.random_split(0.80, seed=107)

In [ ]:
train_train_train, train_train_cv = train_train.random_split(0.75, seed=7)

In [ ]:
TRAIN, CV = train_train_cv.random_split(0.50, seed=113)

In [ ]:
TRAIN.shape, CV.shape

In [ ]:
features

# Generate Submission Model

Started at 06:35:00am

In [ ]:
model = gl.classifier.boosted_trees_classifier.create(train, target='sponsored',
                                                      #features=features + ['tfidf_hashed_18'],
                                                      features=features + ['tfidf5e5'],
                                                      max_depth=6,
                                                      step_size=0.2,
                                                      max_iterations=300,
                                                      column_subsample=0.3,
                                                      row_subsample=1.0,
                                                      class_weights='auto')

In [ ]:
model = gl.classifier.random_forest_classifier.create(train, target='sponsored',
                                                      features=features, # + ['tfidf', 'word2vec'],
                                                      num_trees=600,
                                                      max_depth=150,
                                                      validation_set=None,
                                                      column_subsample=0.3,
                                                      row_subsample=0.85,
                                                      class_weights='auto')

In [ ]:
model = gl.classifier.boosted_trees_classifier.create(train, target='sponsored',
                                                      #features=features + ['tfidf_hashed_18'],
                                                      features=features + ['tfidf'],
                                                      max_depth=6,
                                                      step_size=0.2,
                                                      max_iterations=300,
                                                      column_subsample=0.3,
                                                      row_subsample=1.0,
                                                      class_weights='auto',
                                                      validation_set=None)

In [ ]:
lr_model = gl.logistic_classifier.create(TRAIN_l, target='sponsored', 
                                      features=new_feats,
                                      validation_set=CV_l,
                                      class_weights='auto',
                                      max_iterations=10,
                                      l2_penalty=0.00,
                                      l1_penalty=0.00)

In [ ]:
svm_model = gl.svm_classifier.create(train, target='sponsored', 
                                      features=['tfidf_hashed'],
                                      validation_set=None,                                           
                                      class_weights='auto',
                                      max_iterations=20)

### Output model

In [ ]:
import datetime
print(datetime.datetime.now())

In [ ]:
ypred = model.predict(test, 'probability')

submission = gl.SFrame()
submission['file'] = test['id'].apply(lambda x: x + '_raw_html.txt')
submission['sponsored'] = ypred 
#submission.save('submission_version_4.csv', format='csv')

submission = submission.to_dataframe()
submission.to_csv('submission_rf_ga_2.csv', index=False, float_format='%1.8f')

In [ ]:
model.save('/mnt/sframe/model_RF_200_150_noword2vec')

In [ ]:
test

# Experiment

### Split train into *train_train*/*train_cv*

In [ ]:
model = gl.logistic_classifier.create(train_train, target='sponsored', 
                                      features=features + ['tfidf'],
                                      validation_set=train_cv,
                                      class_weights='auto',
                                      max_iterations=30,
                                      feature_rescaling=True,
                                      l2_penalty=0.00,
                                      l1_penalty=0.00)

In [ ]:
results = gl.SFrame()
results['id'] = train_cv['id']
results['actual'] = train_cv['sponsored']
results['predicted'] = model.predict(train_cv)

In [ ]:
train_cv.unpack('tfidf')

In [ ]:
FN.shape, FP.shape

In [ ]:
FN = results[results['actual'] > results['predicted']]
FP = results[results['actual'] < results['predicted']]

In [ ]:
FN[720:730]

In [ ]:
FP

In [ ]:
model.evaluate(train_cv)

In [ ]:
results = model.evaluate(train_cv, metric='roc_curve')
a = results['roc_curve']

fpr = list(a['fpr'])
tpr = list(a['tpr'])
fpr[0] = 1.0
tpr[0] = 1.0
fpr = np.array(fpr)
tpr = np.array(tpr)

AUC = np.sum((fpr[:-1] - fpr[1:]) * (tpr[:-1] + (tpr[:-1] - tpr[1:])/2))
plt.plot(fpr, tpr)
print('AUC = %f'%AUC)

In [ ]:
train_cv.remove_column('tfidf_hashed17')

In [ ]:
svm_model = gl.svm_classifier.create(train_train, target='sponsored', 
                                      features=features + ['tfidf_hashed'],
                                      validation_set=train_cv,                                           
                                      class_weights='auto',
                                      max_iterations=15)

In [ ]:
svm_model

In [ ]:
train_cv['margin'] = svm_model.predict(train_cv, output_type='margin')
preds = train_cv[['sponsored', 'margin']].sort('margin')
train_cv.remove_column('margin')

pd_preds = preds.to_dataframe()
pd_preds['number'] = 1.0

pd_preds_cum = pd_preds.cumsum()

total_positives = np.asarray(pd_preds_cum['sponsored'])[-1]
total = np.asarray(pd_preds_cum['number'])[-1]
total_negatives = total - total_positives

pd_preds_cum['FN'] = pd_preds_cum['sponsored']
pd_preds_cum['TN'] = pd_preds_cum['number'] - pd_preds_cum['sponsored']

pd_preds_cum['TP'] = total_positives - pd_preds_cum['FN']
pd_preds_cum['FP'] = total - total_positives - pd_preds_cum['TN']

pd_preds_cum['fpr'] = pd_preds_cum['FP'] / (pd_preds_cum['FP'] + pd_preds_cum['TN'])
pd_preds_cum['tpr'] = pd_preds_cum['TP'] / (pd_preds_cum['TP'] + pd_preds_cum['FN'])



In [ ]:
a = pd_preds_cum

fpr = list(a['fpr'])
tpr = list(a['tpr'])
fpr[0] = 1.0
tpr[0] = 1.0
fpr = np.array(fpr)
tpr = np.array(tpr)

AUC = np.sum((fpr[:-1] - fpr[1:]) * (tpr[:-1] + (tpr[:-1] - tpr[1:])/2))
plt.plot(fpr, tpr)
print('AUC = %f'%AUC)

In [ ]:
train_cv['margin'] = svm_model.predict(train_cv, output_type='margin')
preds = train_cv[['sponsored', 'margin']]
preds['margin'].show()

In [ ]:
(preds[preds['margin'] < 55]['sponsored']).sum()

In [ ]:
ts = np.arange(-22, 50, 0.1)
[for t in ts]

In [ ]:
svm_model.evaluate(train_cv)

In [ ]:
results = svm_model.evaluate(train_cv, metric='roc_curve')
a = results['roc_curve']

fpr = list(a['fpr'])
tpr = list(a['tpr'])
fpr[0] = 1.0
tpr[0] = 1.0
fpr = np.array(fpr)
tpr = np.array(tpr)

AUC = np.sum((fpr[:-1] - fpr[1:]) * (tpr[:-1] + (tpr[:-1] - tpr[1:])/2))
plt.plot(fpr, tpr)
print('AUC = %f'%AUC)

In [ ]:
results

# Save Datasets?

In [ ]:
train.save('/mnt/sframe/plain_train')
test.save('/mnt/sframe/plain_test')

In [ ]:
train_train = gl.load_sframe('/mnt/sframe/shinn_split_train_train')
train_cv = gl.load_sframe('/mnt/sframe/shinn_split_train_cv')

In [ ]:
train_train.head()

# Junk

In [ ]:
model_17 = gl.logistic_classifier.create(train_train, target='sponsored', 
                                      features=['tfidf_hashed_17'],
                                      validation_set=train_cv,
                                      class_weights=None, #'auto',
                                      max_iterations=7,
                                      feature_rescaling=True,
                                      l2_penalty=0.00,
                                      l1_penalty=0.00)

In [ ]:
model_17.evaluate(train_cv)

In [ ]:
results = model_17.evaluate(train_cv, metric='roc_curve')
a = results['roc_curve']

fpr = list(a['fpr'])
tpr = list(a['tpr'])
fpr[0] = 1.0
tpr[0] = 1.0
fpr = np.array(fpr)
tpr = np.array(tpr)

AUC = np.sum((fpr[:-1] - fpr[1:]) * (tpr[:-1] + (tpr[:-1] - tpr[1:])/2))
plt.plot(fpr, tpr)
print('AUC = %f'%AUC)

In [ ]:
gl.svm_classifier.create(train_train, target='sponsored', 
                                      features=['tfidf_hashed_15'],
                                      validation_set=train_cv,                                           
                                      class_weights='auto',
                                      max_iterations=40)

# Continue with data

In [ ]:
train = gl.load_sframe('/mnt/sframe/counts_and_tfidf_hashed_18_train')
test = gl.load_sframe('/mnt/sframe/counts_and_tfidf_hashed_18_test')

train_train = gl.load_sframe('/mnt/sframe/num_words_counts_and_tfidf_hashed_18_split_train_train')
train_cv = gl.load_sframe('/mnt/sframe/num_words_counts_and_tfidf_hashed_18_split_train_cv')

TRAIN, CV = train_cv.random_split(0.50, seed=113)

In [ ]:
train.save('/mnt/sframe/shinn_train')
test.save('/mnt/sframe/shinn_test')

train_train.save('/mnt/sframe/shinn_split_train_train')
train_cv.save('/mnt/sframe/shinn_split_train_cv')
#train_train = gl.load_sframe('/mnt/sframe/tfidf_hashed_16_split_train_train')
#train_cv = gl.load_sframe('/mnt/sframe/tfidf_hashed_16_split_train_cv')

## Try classifiers

In [ ]:
def text2wordsetsize(text):
    if not text:
        return 0.0
    else:
        return len(set(text.split()))
    
def safe_divide(a, b):
    if b > 0:
        return a * 1.0 / b
    else:
        return 0.0

TRAIN['word_set_size2'] = TRAIN['text'].apply(text2wordsetsize)
CV['word_set_size2'] = CV['text'].apply(text2wordsetsize)

TRAIN['word_set_size2_ratio'] = TRAIN.apply(lambda r: safe_divide(r['word_set_size2'], r['word_set_size']))
CV['word_set_size2_ratio'] = CV.apply(lambda r: safe_divide(r['word_set_size2'], r['word_set_size']))

In [ ]:
TRAIN.head()

In [ ]:
model_boosted.get_feature_importanceature_importance()

In [ ]:
# model_rf = gl.classifier.random_forest_classifier.create(train_train, target='sponsored',
#                                                       features=features + ['word2vec'],
#                                                       num_trees=10,
#                                                       max_depth=200,
#                                                       column_subsample=0.15,
#                                                       row_subsample=1.0,
#                                                       class_weights='auto',
#                                                       validation_set=train_cv)
model_boosted = gl.classifier.boosted_trees_classifier.create(TRAIN, target='sponsored',
                                                      features=features + ['shinn', # 'word_set_size2', 
                                                                           #'word_set_size2_ratio',
                                                                           'tfidf'], #, 'word2vec'],
                                                      max_depth=2,
                                                      step_size=0.15,  #0.2
                                                      max_iterations=15,
                                                      column_subsample=0.4,
                                                      row_subsample=1.0,
                                                      class_weights='auto',
                                                      validation_set=CV)

In [ ]:
results = model_boosted.evaluate(CV, metric='roc_curve')
a = results['roc_curve']

fpr = list(a['fpr'])
tpr = list(a['tpr'])
fpr[0] = 1.0
tpr[0] = 1.0
fpr = np.array(fpr)
tpr = np.array(tpr)

AUC = np.sum((fpr[:-1] - fpr[1:]) * (tpr[:-1] + (tpr[:-1] - tpr[1:])/2))
plt.plot(fpr, tpr)
print('AUC = %f'%AUC)

In [ ]:
model_rf.get_feature_importance().print_rows(60)

In [ ]:
TRAIN.head()

In [ ]:
features

In [ ]:
train_train['ga_id'] = train_train['ga_id'].apply(float)
train_cv['ga_id'] = train_cv['ga_id'].apply(float)

train_train['ga_subid'] = train_train['ga_subid'].apply(float)
train_cv['ga_subid'] = train_cv['ga_subid'].apply(float)

train_train['fb_id'] = train_train['fb_id'].apply(float)
train_cv['fb_id'] = train_cv['fb_id'].apply(float)

In [ ]:
TRAIN['ga_id'] = TRAIN['ga_id'].apply(float)
CV['ga_id'] = CV['ga_id'].apply(float)

TRAIN['ga_subid'] = TRAIN['ga_subid'].apply(float)
CV['ga_subid'] = CV['ga_subid'].apply(float)

TRAIN['fb_id'] = TRAIN['fb_id'].apply(float)
CV['fb_id'] = CV['fb_id'].apply(float)

In [ ]:
TRAIN.head()

In [ ]:
TRAIN.dtype

In [ ]:
model_rf.get_feature_importance().print_rows(100)

In [ ]:
model_rf.get_feature_importance().print_rows(40)

In [ ]:
model_rf.get_feature_importance().print_rows(40)

In [ ]:
model_rf = gl.classifier.random_forest_classifier.create(train_train, target='sponsored',
                                                      features=features,
                                                      num_trees=100, 
                                                      max_depth=150,
                                                      column_subsample=0.45,
                                                      row_subsample=1.0,
                                                      class_weights='auto',
                                                      validation_set=train_cv)

In [ ]:
TRAIN.shape

In [ ]:
results = model_rf.evaluate(train_cv, metric='roc_curve')
a = results['roc_curve']

fpr = list(a['fpr'])
tpr = list(a['tpr'])
fpr[0] = 1.0
tpr[0] = 1.0
fpr = np.array(fpr)
tpr = np.array(tpr)

AUC = np.sum((fpr[:-1] - fpr[1:]) * (tpr[:-1] + (tpr[:-1] - tpr[1:])/2))
plt.plot(fpr, tpr)
print('AUC = %f'%AUC)

In [ ]:
train_train.head()

In [ ]:
def generate_inverse(dataset, feats):
    new_feats = []
    for f in feats:
        new_feat = 'inv_' + f
        new_feats.append(new_feat)
        dataset[new_feat] = dataset.apply(lambda r: safe_divide(1, r[f]))
    return new_feats            

In [ ]:
features.remove('shinn')

In [ ]:
features

In [ ]:
new_feats = generate_inverse(TRAIN, features)
generate_inverse(CV, features)

In [ ]:
new_feats + features

quadratic = gl.feature_engineering.QuadraticFeatures(features=features + new_feats)
fit_quadratic = quadratic.fit(TRAIN)
TRAIN = fit_quadratic.transform(TRAIN)
CV = fit_quadratic.transform(CV)


In [ ]:
a = TRAIN.column_names()

In [ ]:
def for_inverse(col):
    return col not in set(['sponsored', 'id', 'text', 'text_words', 'words', 'bow', 'tfidf', 'shinn'])
[x for x in .column_names() if for_inverse(x)]

In [ ]:
TRAIN.remove_columns?

In [ ]:
T_TRAIN = TRAIN['word2vec'].unpack()
T_CV = CV['word2vec'].unpack()

T_TRAIN['sponsored'] = TRAIN['sponsored']
T_CV['sponsored'] = CV['sponsored']

In [ ]:
model_nn = gl.nearest_neighbor_classifier.create(TRAIN, target='sponsored', 
                                                 features=['tfidf'] # 'word2vec']
                                                )

In [ ]:
preds = model_nn.predict(CV, max_neighbors=200, output_type='probability')

In [ ]:
preds

In [ ]:
T_CV['prob'] = preds
preds = T_CV[['sponsored', 'prob']].sort('prob')
T_CV.remove_column('prob')

pd_preds = preds.to_dataframe()
pd_preds['number'] = 1.0

pd_preds_cum = pd_preds.cumsum()

total_positives = np.asarray(pd_preds_cum['sponsored'])[-1]
total = np.asarray(pd_preds_cum['number'])[-1]
total_negatives = total - total_positives

pd_preds_cum['FN'] = pd_preds_cum['sponsored']
pd_preds_cum['TN'] = pd_preds_cum['number'] - pd_preds_cum['sponsored']

pd_preds_cum['TP'] = total_positives - pd_preds_cum['FN']
pd_preds_cum['FP'] = total - total_positives - pd_preds_cum['TN']

pd_preds_cum['fpr'] = pd_preds_cum['FP'] / (pd_preds_cum['FP'] + pd_preds_cum['TN'])
pd_preds_cum['tpr'] = pd_preds_cum['TP'] / (pd_preds_cum['TP'] + pd_preds_cum['FN'])



In [ ]:
a = pd_preds_cum

fpr = list(a['fpr'])
tpr = list(a['tpr'])
fpr[0] = 1.0
tpr[0] = 1.0
fpr = np.array(fpr)
tpr = np.array(tpr)

AUC = np.sum((fpr[:-1] - fpr[1:]) * (tpr[:-1] + (tpr[:-1] - tpr[1:])/2))
plt.plot(fpr, tpr)
print('AUC = %f'%AUC)

In [ ]:
model1 = gl.classifier.random_forest_classifier.create(train_train, target='sponsored',
                                                      features=features + ['tfidf'], #, 'word2vec'],
                                                      num_trees=90, #100,
                                                      max_depth=150,
                                                      column_subsample=0.45,
                                                      row_subsample=1.0,
                                                      class_weights='auto',
                                                      validation_set=train_cv)

In [ ]:
results = model1.evaluate(train_cv, metric='roc_curve')
a = results['roc_curve']

fpr = list(a['fpr'])
tpr = list(a['tpr'])
fpr[0] = 1.0
tpr[0] = 1.0
fpr = np.array(fpr)
tpr = np.array(tpr)

AUC = np.sum((fpr[:-1] - fpr[1:]) * (tpr[:-1] + (tpr[:-1] - tpr[1:])/2))
plt.plot(fpr, tpr)
print('AUC = %f'%AUC)

In [ ]:
model = gl.classifier.random_forest_classifier.create(train_train, target='sponsored',
                                                      features=features + ['tfidf'], #, 'word2vec'],
                                                      num_trees=10,
                                                      max_depth=150,
                                                      column_subsample=0.45,
                                                      row_subsample=1.0,
                                                      class_weights='auto',
                                                      validation_set=train_cv)

In [ ]:
results = model.evaluate(train_cv, metric='roc_curve')
a = results['roc_curve']

fpr = list(a['fpr'])
tpr = list(a['tpr'])
fpr[0] = 1.0
tpr[0] = 1.0
fpr = np.array(fpr)
tpr = np.array(tpr)

AUC = np.sum((fpr[:-1] - fpr[1:]) * (tpr[:-1] + (tpr[:-1] - tpr[1:])/2))
plt.plot(fpr, tpr)
print('AUC = %f'%AUC)

In [ ]:
model.get_feature_importance().print_rows(num_rows=30, num_columns=2) 

In [ ]:
train_train['tfidf_nonone'] = train_train['tfidf'].apply(lambda x: x if x else {})
train_cv['tfidf_nonone'] = train_cv['tfidf'].apply(lambda x: x if x else {})

In [ ]:
train_train['tfidf_nonone'] = train_train['tfidf_nonone'].fillna({})
train_cv['tfidf_nonone'] = train_cv['tfidf_nonone'].fillna({})

In [ ]:
svm_model = gl.svm_classifier.create(train_train, target='sponsored', 
                                      features=features + ['tfidf_nonone'], #features + ['tfidf'],
                                      validation_set=train_cv,                                           
                                      class_weights='auto',
                                      max_iterations=40)

In [ ]:
train_cv['margin'] = svm_model.predict(train_cv, output_type='margin')
preds = train_cv[['sponsored', 'margin']].sort('margin')
train_cv.remove_column('margin')

pd_preds = preds.to_dataframe()
pd_preds['number'] = 1.0

pd_preds_cum = pd_preds.cumsum()

total_positives = np.asarray(pd_preds_cum['sponsored'])[-1]
total = np.asarray(pd_preds_cum['number'])[-1]
total_negatives = total - total_positives

pd_preds_cum['FN'] = pd_preds_cum['sponsored']
pd_preds_cum['TN'] = pd_preds_cum['number'] - pd_preds_cum['sponsored']

pd_preds_cum['TP'] = total_positives - pd_preds_cum['FN']
pd_preds_cum['FP'] = total - total_positives - pd_preds_cum['TN']

pd_preds_cum['fpr'] = pd_preds_cum['FP'] / (pd_preds_cum['FP'] + pd_preds_cum['TN'])
pd_preds_cum['tpr'] = pd_preds_cum['TP'] / (pd_preds_cum['TP'] + pd_preds_cum['FN'])

#   show
a = pd_preds_cum

fpr = list(a['fpr'])
tpr = list(a['tpr'])
fpr[0] = 1.0
tpr[0] = 1.0
fpr = np.array(fpr)
tpr = np.array(tpr)

AUC = np.sum((fpr[:-1] - fpr[1:]) * (tpr[:-1] + (tpr[:-1] - tpr[1:])/2))
plt.plot(fpr, tpr)
print('AUC = %f'%AUC)

In [ ]:
model_boosted = gl.classifier.boosted_trees_classifier.create(train_train, target='sponsored',
                                                      features=features + ['tfidf_hashed_18'],
                                                      max_depth=6,
                                                      step_size=0.2,
                                                      max_iterations=300,
                                                      column_subsample=0.3,
                                                      row_subsample=1.0,
                                                      class_weights='auto',
                                                      validation_set=train_cv)

In [ ]:
results = model_boosted.evaluate(train_cv, metric='roc_curve')
a = results['roc_curve']

fpr = list(a['fpr'])
tpr = list(a['tpr'])
fpr[0] = 1.0
tpr[0] = 1.0
fpr = np.array(fpr)
tpr = np.array(tpr)

AUC = np.sum((fpr[:-1] - fpr[1:]) * (tpr[:-1] + (tpr[:-1] - tpr[1:])/2))
plt.plot(fpr, tpr)
print('AUC = %f'%AUC)

In [ ]:
gl.boosted_trees_classifier.get_default_options()

In [ ]:
TRAIN, CV = train_cv.random_split(0.50, seed=113)

In [ ]:
CV.shape

In [ ]:
model_boosted = gl.classifier.boosted_trees_classifier.create(TRAIN, target='sponsored',
                                                      features=features + ['tfidf_hashed_18'],
                                                      max_depth=6,
                                                      step_size=0.2,
                                                      max_iterations=500,
                                                      column_subsample=0.3,
                                                      row_subsample=1.0,
                                                      class_weights='auto',
                                                      validation_set=CV)

In [ ]:
results = model_boosted.evaluate(CV, metric='roc_curve')
a = results['roc_curve']

fpr = list(a['fpr'])
tpr = list(a['tpr'])
fpr[0] = 1.0
tpr[0] = 1.0
fpr = np.array(fpr)
tpr = np.array(tpr)

AUC = np.sum((fpr[:-1] - fpr[1:]) * (tpr[:-1] + (tpr[:-1] - tpr[1:])/2))
plt.plot(fpr, tpr)
print('AUC = %f'%AUC)

In [ ]:
model_boosted = gl.classifier.boosted_trees_classifier.create(train_train, target='sponsored',
                                                      features=features + ['tfidf_hashed_18'],
                                                      max_depth=6,
                                                      step_size=0.2,
                                                      max_iterations=400,
                                                      column_subsample=0.4,
                                                      row_subsample=1.0,
                                                      class_weights='auto',
                                                      validation_set=train_cv)

In [ ]:
train_cv_pred = gl.SFrame()
train_cv_pred['pred'] =model_boosted.predict(train_cv)
train_cv_pred['actual'] = train_cv['sponsored']
train_cv_pred['id'] = train_cv['id']

In [ ]:
pred_over = train_cv_pred[train_cv_pred['pred'] > train_cv_pred['actual']]
pred_under = train_cv_pred[train_cv_pred['pred'] < train_cv_pred['actual']]

In [ ]:
for z in list(pred_under.sample(0.02)['id'].apply(lambda x: 'aws s3 cp s3://sparkydotsdata/kaggle/native/orig/' + x + '_raw_html.txt ' + x +'raw_html')):
    print(z) 

In [ ]:
results = model_boosted.evaluate(train_cv, metric='roc_curve')
a = results['roc_curve']

fpr = list(a['fpr'])
tpr = list(a['tpr'])
fpr[0] = 1.0
tpr[0] = 1.0
fpr = np.array(fpr)
tpr = np.array(tpr)

AUC = np.sum((fpr[:-1] - fpr[1:]) * (tpr[:-1] + (tpr[:-1] - tpr[1:])/2))
plt.plot(fpr, tpr)
print('AUC = %f'%AUC)

In [ ]:
def custom_evaluator(model, train, test):
    results = model.evaluate(test, metric='roc_curve')
    a = results['roc_curve']

    fpr = list(a['fpr'])
    tpr = list(a['tpr'])
    fpr[0] = 1.0
    tpr[0] = 1.0
    fpr = np.array(fpr)
    tpr = np.array(tpr)

    AUC = np.sum((fpr[:-1] - fpr[1:]) * (tpr[:-1] + (tpr[:-1] - tpr[1:])/2))
    return {'AUC': AUC}

In [ ]:
job0 = job

In [ ]:
params = dict([
        ('target', 'sponsored'),
        ('features', [features + ['tfidf_hashed_18']]),
        ('max_depth', [6]),
        ('step_size', [0.2]),
        ('max_iterations', [100, 150, 200]),
        ('column_subsample', [0.4]),
        ('validation_set', [None])
    ])

job = gl.grid_search.create((TRAIN, CV), 
                              gl.boosted_trees_classifier.create, 
                              params, 
                              evaluator=custom_evaluator)
job.get_results()

In [ ]:
models = job.get_models()

In [ ]:
results = job.get_results()

In [ ]:
results = results.to_dataframe()

In [ ]:
results.boxplot('AUC', by='max_iterations')

In [ ]:
results.sort('AUC', ascending=False)

In [ ]:
len(models)

In [ ]:
job.get_metrics()

In [ ]:
aa = scipy.stats.distributions.expon(.1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

In [ ]:
clf = RandomForestClassifier(n_estimators=10, n_jobs=-1, random_state=0)
#train = df_full[df_full.sponsored.notnull()].fillna(0)
#test = df_full[df_full.sponsored.isnull() & df_full.file.isin(test_files)].fillna(0)
# clf.fit(train.drop(['file', 'sponsored'], 1), train.sponsored)

# print('--- Create predictions and submission')
# submission = test[['file']].reset_index(drop=True)
# submission['sponsored'] = clf.predict_proba(test.drop(['file', 'sponsored'], 1))[:, 1]
# submission.to_csv('native_btb_basic_submission.csv', index=False)

In [ ]:
shiTRAIN = shiTRAIN.to_dataframe()
shiCV = shiCV.to_dataframe()

In [ ]:
for col in shiTRAIN.column_names:
    shiTRAIN[col] = 

In [ ]:
clf.fit(shiTRAIN, shiTRAIN_label)

In [ ]:
shiTRAIN = TRAIN.unpack('shinn')
shiCV = CV.unpack('shinn')

In [ ]:
shiTRAIN_label = np.asarray(shiTRAIN['sponsored'])
shiCV_label = np.asarray(shiCV['sponsored'])

In [ ]:
shiTRAIN_label = np.asarray(shiTRAIN_label, float)
shiCV_label = np.asarray(shiCV_label, float)

In [ ]:
shiTRAIN_tf = shiTRAIN['tfidf5e5']
shiCV_tf = shiCV['tfidf5e5']

In [ ]:
shiTRAIN.remove_columns(['tfidf5e5'])
shiCV.remove_columns(['tfidf5e5'])
# shiTRAIN.remove_columns(['text', 'bow', 'sponsored', 'id'])
# shiCV.remove_columns(['text', 'bow', 'sponsored', 'id'])

In [ ]:
shiTRAIN = shiTRAIN.to_dataframe drop('text', 1)
shiCV = shiCV.drop('text', 1)

In [ ]:
shiTRAIN.head()